In [ ]:
import pyvo

"""Get information of stars in the Orion Cluster from SIMBAD"""
# Make the TAP service object
service = pyvo.dal.TAPService ("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

# Query the TAP service with a simple ADQL query.
stars = service.search ("""
SELECT
   distinct main_id,ra, dec, cats.id
    FROM ident AS orion,basic 
    JOIN ident AS cats 
    ON basic.oid=cats.oidref 
    JOIN h_link ON (oid=child 
    AND parent=orion.oidref)
    WHERE orion.id = 'Orion Cluster' 
    AND membership >=90
    AND cats.id LIKE 'Gaia DR3 %' 
    AND basic.otype = 'V*..' 

  """)



In [ ]:
"""X-match with the ESA gaia DR3 catalog"""

esa_service = pyvo.dal.TAPService ("https://gea.esac.esa.int/tap-server/tap")

orion = esa_service.search ("""
    SELECT
    
    simbad.main_id, source_id, designation, 
    phot_variable_flag,phot_g_mean_mag, 
    phot_bp_mean_mag, phot_rp_mean_mag, 
    gaia.ra, gaia.dec,parallax, pm, 
    pmra, pmdec, radial_velocity 
    
    FROM gaiadr3.gaia_source AS gaia
    
    JOIN TAP_UPLOAD.simbad AS simbad
    ON simbad.id=designation
    
    WHERE has_epoch_photometry ='t'
    """, uploads={'simbad':stars})

In [ ]:
import matplotlib.pyplot as plt

def draw_plot(to_plot, 
                colours  = ['green', 'red', 'blue'], 
                title = 'Epoch photometry', 
                fontsize = 12, 
                show_legend = True, 
                show_grid = True, 
                figsize = [15,5]):
    """
    Epoch photometry plotter. 'inp_table' MUST be an Astropy-table object.
    """
    (title,inp_table,cols) = to_plot
    xcol, ycol, col_p = cols
    fig  = plt.figure(figsize=figsize)
    xlabel   = f'JD date [{inp_table[xcol].unit}]'
    ylabel   = f'magnitude [{inp_table[ycol].unit}]'
    gbands   = ['G', 'RP', 'BP']
    colours  = iter(colours)

    plt.gca().invert_yaxis()
    for band in gbands:
        phot_set = inp_table[inp_table[col_p] == band]
        plt.plot(phot_set[xcol], 
                 phot_set[ycol], 
                 'o', 
                 label = band, 
                 color = next(colours))

    make_canvas(title = title, 
                xlabel = xlabel, 
                ylabel = ylabel, 
                fontsize= fontsize, 
                show_legend=show_legend, 
                show_grid = show_grid)
    return


def draw_spec_plot(spec_to_plot, 
                colours  = ['blue'], 
                title = 'Gaia Spectrum', 
                fontsize = 12, 
                show_legend = True, 
                show_grid = True, 
                figsize = [15,5]):
    """
    Spectrum plot 'inp_table' MUST be an Astropy-table object.
    """
    (title,inp_table,cols) = spec_to_plot
    xcol, ycol = cols
    fig  = plt.figure(figsize=figsize)
    xlabel   = f'Spectral [{inp_table[xcol].unit}]'
    ylabel   = f'Flux [{inp_table[ycol].unit}]'
    colours  = colours

    plt.plot(inp_table[xcol], 
             inp_table[ycol], 
             'o')

    make_canvas(title = title, 
                xlabel = xlabel, 
                ylabel = ylabel, 
                fontsize= fontsize, 
                show_legend=show_legend, 
                show_grid = show_grid)
    return



def make_canvas(
                title ='',
                xlabel = '', 
                ylabel = '', 
                show_grid = False, 
                show_legend = False, 
                fontsize = 12):
    """
    Create generic canvas for plots
    """

    plt.title(title,
          fontsize = fontsize)
    plt.xlabel(xlabel, 
           fontsize = fontsize)
    plt.ylabel(ylabel, 
           fontsize = fontsize)
    plt.xticks(fontsize = fontsize)
    plt.yticks(fontsize = fontsize)

    if show_grid:
        plt.grid()
    if show_legend:
        plt.legend(fontsize = fontsize*0.75)

class TableIndex():
    
    def forward (self):
        self.index+=1
    def rewind (self):
        self.index-=1    
    def __init__(self):
        self.index=0
        
class Spectrum():
    def update(self):
        self.spec=False
    def incspec(self, spec):
        self.spec=spec
    def __init__(self):
        self.name=""
        self.spec=False
    

In [ ]:
# ipywidets gives us control over java script from the jupyter notebook. We use this for the buttons. 


import ipywidgets as widgets
import astropy.units as units

from astropy.units import Quantity
from IPython.display import clear_output
from IPython.display import display

from astropy.io.votable import from_table
from astropy.coordinates import SkyCoord
from astropy.io.votable import parse

import ipyaladin.aladin_widget as ipyal
from astroquery.cds import cds


index=TableIndex()
ssa=Spectrum()
survey = cds.find_datasets(meta_data="ID=*P/XMM*")['hips_service_url_1'][0]

aladin = ipyal.Aladin(
    survey=survey,
    show_coo_grid=True,
    target='galactic center',
    coo_frame="galactic",
    fov=0.2 )

def get_url_from_datalink(row):
    """From the datalink make the URL to the lightcurve"""
    
    dl=row.getdatalink()
 
    # There may be more than one datalink, so get the one with the lc of g band 
    for i in dl:
        if i['description'].find('Epoch photometry')!=-1:
            # Use the access_url attribut
            accurl=str(i['access_url'])           
    return accurl

def get_lightcurve(accurl):
    
    # Parse the VOTable from the link -- we have the lightcurve
    lc_vot=parse(accurl)
    
    return lc_vot

def get_lctable(row):
    "Get the lightvurves through the datalink"
    
    # First make the url of the lightcurve
    url=get_url_from_datalink(row)
    
    # Then download the lightcurve table from the URL
    lctable=get_lightcurve(url)
    
    return lctable

def get_colname_from_ucd(fields, ucdpattern):
    """
    Searches through the fields of an astropy votable object and returns
    the list of matches
    """

    matches=[]

    for field in fields:
        if field.ucd:
            ucd=field.ucd.lower()
            if ucd.find(ucdpattern.lower())!=-1:
                matches.append(field.name)
    return matches

def get_colnames(lc_vot):
    """Use UCDs to make the column names. 
        Input is the lightcurve table of a source"""
    ucds=['time.epoch','phot.mag;em.opt', 'instr.bandpass']
    cols=[]
    for ucd in ucds:
        cols.append(get_colname_from_ucd(lc_vot.fields, ucd)[0])
    return tuple(cols)

def make_coord(row):
    """Make the coordinates object from the position
        will be used for a Simple Access Protocol"""
    coord=SkyCoord(row['ra']*units.deg, row['dec']*units.deg, frame='icrs')
    
    return coord

def search_spectrum(coord):
    """ Gaia DR3 MC sampled XP spectra SSA from GAVO DC """
    ssa_service = pyvo.dal.SSAService("http://dc.zah.uni-heidelberg.de/gaia/s3/ssa/ssap.xml?")
    ssa_results = ssa_service.search(pos = coord, diameter=Quantity(3, unit="arcsec"))
    
    return ssa_results


"""The following functions are for making the buttons to navigate the table
    and to send the spectrum to Cassis."""

def make_buttons ():
    bns={}
    
    # Button previous
    bn=widgets.Button(description = 'Previous')
    bn.on_click(previous_row)        
    bns['prev']=bn
    
    # Button Next
    bn=widgets.Button(description = 'Next')
    bn.on_click(next_row) 
    bns['next']=bn
    
    # Button Spec
    bn=widgets.Button(description = 'Show Spec')
    bn.on_click(show_spec)
    bns['dlspec']=bn
    
    return bns
    

def make_hbox(orion, ind, nav_btns, ssa):

    bns=[]
    # If there is a previous data record, display Previous
    if ind != 0:
        bns.append(nav_btns['prev'])

    # If there is a next data record, display Next
    if ind != len(orion)-1:
        bns.append(nav_btns['next'])

    # If a spectrum is available, display Button to send it to Cassis    
    if ssa.spec != False:
        bns.append(nav_btns['dlspec'])
    
    return widgets.HBox(bns)


def show_spec(btns, ssa=ssa):
    
    acref=ssa.spec[0].acref
    spec=parse(acref).get_first_table().to_table()
    to_plot=(ssa.name,  spec, ('spectral', 'flux'))    
    fig2 = draw_spec_plot
    
    output = widgets.Output()  
    with output:
        display (fig2(to_plot))
#    ssa.spec[0].
    pass

def previous_row(btn, orion=orion, aladin=aladin, index=index):
    index.rewind()
    btn_action(orion, aladin, index)

def next_row(btn, orion=orion, aladin=aladin, index=index):
    index.forward()
    btn_action(orion, aladin, index)

    
NAV_BUTTONS=make_buttons()    



def start_sciplat(orion, aladin):
    
    index=TableIndex()
    ssa=Spectrum()
    btn_action(orion, aladin, index)

def btn_action (orion, aladin, index):
    
    ssa.update()
    ind=index.index
    nav_buttons=NAV_BUTTONS
    
    row=orion[ind]
    lctable=get_lctable(row)
    name=row['main_id']

    ssa_results=search_spectrum(make_coord(row))
    if len(ssa_results)>=1:
        ssa.incspec(ssa_results)
        ssa.name=name

    else:
        ssa.spec=False
  
    # get colnames from ucds and table
    colnames=get_colnames(lctable.get_first_table())

    
    to_plot=(name, lctable.get_first_table().to_table(), colnames)
    
    output = widgets.Output()
                   
    # Update Aladin Focus
    aladin.target=name
    aladin.fov=0.2
    
    clear_output()

    fig = draw_plot
    hbox1 = make_hbox(orion, ind, nav_buttons, ssa)
    
    display(hbox1)


    with output:
        display (fig(to_plot))

        



In [ ]:
# create Aladin view
aladin



In [ ]:
# Run the poor man's science platform
start_sciplat (orion, aladin)
